<div align="center">

<h3 style="color:#3A86FF;">📊 PyNext Final Assignment — EDA & Insights</h3>
<h5 style="color:#FF006E;">Submitted by: <b>Ishat Noor Mahi</b></h5>
<h6 style="color:#8338EC;">ID: <b>Mahi-3006</b></h6>
<h6 style="color:#FB5607;">Dept: <b>STATISTICS | 2024-2025</b></h6>

</div>


<div align="center">
    <h5 style="color:#FFBE0B;"><b>1. Data Audit &amp; Cleaning (15 marks)</b></h5>
</div>


In [1]:
import numpy as np
import pandas as pd

In [69]:
#i. Load the dataset; print rows × columns and check student_id uniqueness.
path = 'C:/Users/DELL/PyNext/Datasets/pynext_simple_gradebook - pynext_simple_gradebook.csv' 
gradebook = pd.read_csv(path)

print(gradebook.shape)
#print(gradebook.columns) #to get the column names correctly

print(gradebook['student_id'].nunique())

gradebook.drop_duplicates(subset=['student_id'],keep='first',inplace=True)

print(gradebook.shape)

(306, 36)
300
(300, 36)


In [ ]:
#ii. Show dtypes and % missing for each column.
info_df = pd.DataFrame(
    {
        'Column' : gradebook.columns,
        'Data_type': gradebook.dtypes.values,
        '%Missing_Value': ((gradebook.isna().sum()/len(gradebook)).round(4)*100).values
    },index=np.arange(1,len(gradebook.columns)+1)
)
print(info_df)
print(len(gradebook))

In [ ]:
#iii. Normalize dept and gender to clean categories.

gradebook.groupby('dept')['dept'].count()

def normalize_dept(x):
    x = x.astype(str)
    x = str.lower().str.strip()

    mapping = {
        'c.s.e':'CSE','cse':'CSE',
        'e.e.e':'EEE','eleceng':'EEE','eee':'EEE',
        'ipe':'IPE',
        'mat':'MAT','mathematics':'MAT',
        'mee':'MEE',
        'phy':'PHY','physics':'PHY',
        'sta':'STA','statistics':'STA',
        'swe':'SWE','softwareeng':'SWE',      
    }

    return x.replace(mapping).str.upper()

gradebook['dept_clean'] = normalize_dept(gradebook['dept'])

print(gradebook.groupby('dept_clean')['student_id'].count())


In [ ]:
#iii. Normalize dept and gender to clean categories.


def normalize_gender(x):

    x = x.astype(str)
    x = x.str.strip().str.lower()

    mapping = {
        'f':'Female','female':'Female',
        "m":'Male', 'male':'Male',
        'other':'Other'
    }

    return x.replace(mapping)

gradebook['gender_clean'] = normalize_gender(gradebook['gender'])

gradebook.groupby('gender_clean')['student_id'].count()


In [67]:
#iv. Detect outliers in height_cm and weight_kg using IQR; flag them.

def get_outliers(df,col):

    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)

    IQR = Q3 - Q1

    upper_bound = Q3 + 1.5*IQR
    lower_bound = Q1 - 1.5*IQR

    mask = (col<lower_bound) | (col>upper_bound)
    flag_col = f"{col.name}_outlier_flag"
    df[flag_col] = mask
    outliers = df.loc[mask, [col.name, flag_col]]

    result = (
        f"\nOutlier Detection for '{col.name}':\n"
        f"----------------------------------\n"
        f"Total Outliers Found: {mask.sum()}\n\n"
        f"Outlier Rows:\n{outliers.to_string(index=False)}\n"
    )
    
    return result

In [ ]:
hcm = gradebook['height_cm']
wkg = gradebook['weight_kg']

print(get_outliers(gradebook,hcm))
print(get_outliers(gradebook,wkg))

<div align="center">
    <h5 style="color:#FFBE0B;"><b>12. Student-Level Summary (20 marks)</b></h5>
</div>


In [ ]:
#i. Attended Sessions → The total number of sessions a student was present in.
